# ハルカ 小遙

## sp計算方式 part2 - 未進化 L.v10~24

去掉 power up/down 的值

In [12]:
import sqlite3
import pandas as pd
import datetime

class SamePokemon():
    def __init__(self):
        self.conn = sqlite3.connect("../pokemon_database.db")
        self.cursor = self.conn.cursor()
        self.cursor.execute('SELECT * FROM pokemon')
        self.data = self.cursor.fetchall()

    def get_data_as_dataframe(self):
        columns = [
            "id",
            "img_num",
            "name",
            "sp",
            "expertise",
            "level",
            "help_fruit",
            "help_fruit_num",
            "help_ingredient_1",
            "help_ingredient_num_1",
            "help_ingredient_2",
            "help_ingredient_num_2",
            "help_ingredient_3",
            "help_ingredient_num_3",
            "help_time",
            "help_max",
            "skill_main",
            "skill_main_num",
            "skill_main_level",
            "skill_second_1",
            "skill_second_num_1",
            "skill_second_2",
            "skill_second_num_2",
            "skill_second_3",
            "skill_second_num_3",
            "skill_second_4",
            "skill_second_num_4",
            "skill_second_5",
            "skill_second_num_5",
            "power_up",
            "power_down",
            "備註"
        ]

        df = pd.DataFrame(self.data, columns=columns)
        return df

# 創建一個物件
pokemon_instance = SamePokemon()

# 將資料以 DataFrame 格式讀取
pokemon_data = pokemon_instance.get_data_as_dataframe()

# 取得時間字串
time_strings = pokemon_data['help_time']

def convert_to_time_format(time_str):
    # 將時間字串轉換為時間物件
    time_obj = datetime.datetime.strptime(time_str, '%H:%M:%S').time()
    # 將時間物件轉換為整數，以時 * 10000 + 分 * 100 + 秒格式表示
    return time_obj.hour * 10000 + time_obj.minute * 100 + time_obj.second

# 將時間字串轉換為整數格式
time_converted = [convert_to_time_format(time_str) for time_str in time_strings]

# 更新 'help_time' 欄位
time_indices =list(range(7, 8))+list(range(9, 11))+ list(range(19, 21))+list(range(22, 23))+list(range(31, 32))+ list(range(44, 46))+list(range(47, 48))+ list(range(91, 99))

for i, idx in enumerate(time_indices):
    pokemon_data.at[idx, 'help_time'] = time_converted[i]


In [13]:
# 取y值
y = pokemon_data.iloc[list(range(7, 8))+list(range(9, 11))+ list(range(19, 21))+list(range(22, 23))+list(range(31, 32))+ list(range(44, 46))+list(range(47, 48))+ list(range(91, 99)), 3]

In [14]:
#取X值
X = pokemon_data.iloc[list(range(7, 8))+list(range(9, 11))+ list(range(19, 21))+list(range(22, 23))+list(range(31, 32))+ list(range(44, 46))+list(range(47, 48))+ list(range(91, 99)), [5, 7, 9] + list(range(14,21))]

In [15]:
# 對 skill_main 進行 Label Encoding
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
X['skill_main_encoded'] = label_encoder.fit_transform(X['skill_main'])
# 除去原始的 skill_main 欄位
X = X.drop('skill_main', axis=1)


label_encoder = LabelEncoder()
X['skill_second_1_encoded'] = label_encoder.fit_transform(X['skill_second_1'])
# 除去原始的 skill_second_1 欄位
X = X.drop('skill_second_1', axis=1)

In [ ]:
label_encoder = LabelEncoder()
X['skill_second_2_encoded'] = label_encoder.fit_transform(X['skill_second_2'])
# 除去原始的 skill_second_2 欄位
X = X.drop('skill_second_2', axis=1)

label_encoder = LabelEncoder()
X['skill_second_3_encoded'] = label_encoder.fit_transform(X['skill_second_3'])
# 除去原始的 skill_second_3 欄位
X = X.drop('skill_second_3', axis=1)

label_encoder = LabelEncoder()
X['skill_second_4_encoded'] = label_encoder.fit_transform(X['skill_second_4'])
# 除去原始的 skill_second_4 欄位
X = X.drop('skill_second_4', axis=1)

label_encoder = LabelEncoder()
X['skill_second_5_encoded'] = label_encoder.fit_transform(X['skill_second_5'])
# 除去原始的 skill_second_5 欄位
X = X.drop('skill_second_5', axis=1)

## 把數據分割成訓練集&測試集

In [16]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.5,random_state=2)
# test_size=0.3 = 隨意抓取30%測試集
# random_state 重跑訓練值幾遍

In [17]:
from sklearn.linear_model import LinearRegression 
regressor=LinearRegression() # 初始化
model=regressor.fit(X_train,y_train)

In [18]:
y_predict=regressor.predict(X_test)

from sklearn.metrics import r2_score # 2的平方越大(越接近1)，多元線性回歸就越好
score1=r2_score(y_test,y_predict) # 比較訓練集&測試集之間的預測好不好

In [19]:
score1

0.17582205862440448

In [20]:
model.coef_ # y=b+7.03471165e*data1+1.20332163e*data2+3.96425367e*data3+1.53936312e*data4 -4.80880533e*data5

array([-1.03150904e+01, -9.43941096e+01,  1.05077891e+01,  2.39373054e-02,
        1.39038499e+01,  3.92981498e-02,  4.12716572e+01, -1.90861155e+00,
       -5.93777192e+01, -6.39761684e+01])

In [21]:
model.intercept_ #b=74.48195300240218

789.6672509774364

In [22]:
model.score(X,y) # 再次評估loss值 =>跟score1比較=>數據可用

0.5934628445041512